<a href="https://colab.research.google.com/github/toraaglobal/deeplearning/blob/pytorch/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning - Fine-tuning



## Fine Tuning

In [1]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

trainset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=True, download=True,transform=transform)
testset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=False, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

Extracting /root/.pytorch/CIFAR10/cifar-10-python.tar.gz to /root/.pytorch/CIFAR10


In [5]:
for images, labels in trainloader:
  print(images.size(), labels.size())
  break

torch.Size([64, 3, 224, 224]) torch.Size([64])


In [6]:
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
for param in model.parameters():
  param.requires_grad = False

In [8]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [9]:
for i in range(0,7):
  model.classifier[i].requires_grad = True

In [10]:
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096,512),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(512,10),
                      nn.LogSoftmax(dim=1)
                      )


In [11]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=10, bias=True)
    (4): LogSoftmax(dim=1)
  )
)

In [12]:
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [13]:
criterion = nn.NLLLoss()

In [14]:
from torch.optim import Adam

model = model.to(device)
optimizer = Adam(model.parameters())


## Training from the Fully Connected Network onwards

### Re-training the model

In [15]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 2.35013484954834
Epoch(0/1 : Batch number(2/782) : Batch loss : 2.1753339767456055
Epoch(0/1 : Batch number(3/782) : Batch loss : 1.792695164680481
Epoch(0/1 : Batch number(4/782) : Batch loss : 1.7543681859970093
Epoch(0/1 : Batch number(5/782) : Batch loss : 1.6959620714187622
Epoch(0/1 : Batch number(6/782) : Batch loss : 1.368120551109314
Epoch(0/1 : Batch number(7/782) : Batch loss : 1.4511456489562988
Epoch(0/1 : Batch number(8/782) : Batch loss : 1.1988319158554077
Epoch(0/1 : Batch number(9/782) : Batch loss : 1.237826943397522
Epoch(0/1 : Batch number(10/782) : Batch loss : 1.0372555255889893
Epoch(0/1 : Batch number(11/782) : Batch loss : 1.1193467378616333
Epoch(0/1 : Batch number(12/782) : Batch loss : 1.2607128620147705
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.8814897537231445
Epoch(0/1 : Batch number(14/782) : Batch loss : 1.1461851596832275
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.834740400314331
Epoch(0/1 

### The accuracy of the model

In [16]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 80.0% 


## Un-freezing & training on the LAST CNN block onwards

### Re-training the model

In [17]:
for i in range(24,31):
  model.features[i].requires_grad = True


In [18]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 0.4089019298553467
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.5005176067352295
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.6058690547943115
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.3109970986843109
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.4995884895324707
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.3389025628566742
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.5208551287651062
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.5225297212600708
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.6349907517433167
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.49409571290016174
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.3541323244571686
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.41315263509750366
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.5440112948417664
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.4065741300582886
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.6983069181442261
Ep

### The accuracy of the model

In [19]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 81.25% 


## Un-freezing & training on the LAST TWO CNN block onwards

### Re-training the model

In [20]:
for i in range(17,24):
  model.features[i].requires_grad = True

In [21]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 0.3913610279560089
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.42316871881484985
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.400359183549881
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.5457596182823181
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.2832745909690857
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.35378512740135193
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.1766679733991623
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.27584272623062134
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.2657063901424408
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.29376426339149475
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.3128446936607361
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.3556897044181824
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.19641174376010895
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.3157234191894531
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.2650260627269745


### The accuracy of the model

In [22]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 82.1875% 
